In [7]:
import tensorflow as tf
import numpy as np
import import_ipynb
import cbow_batch_generator

In [8]:
batch_size=8
learning_rate=0.5
window_size = 1
vector_len = 50
vocab_size = 33702
neg_sample=64
steps = 150000

val_indexes = [445,633,4904,16842,1478]

In [9]:
def cbow():
    Xs = tf.placeholder(tf.int32 , [batch_size , window_size*2])
    Ys = tf.placeholder(tf.int32 , [batch_size , 1])
    
    with tf.variable_scope('cbowWord2Vec') as scope:
        Vs = tf.Variable(tf.random_uniform([vocab_size , vector_len],-1.0,1.0))
        Us = tf.Variable(tf.truncated_normal([vocab_size , vector_len] , stddev=1.0/(vector_len**0.5)))
        
        embds = tf.nn.embedding_lookup(Vs , Xs)
        embds = tf.reduce_mean(embds , axis = 1)
        
        nce_biases = tf.Variable(tf.zeros([vocab_size]))
        
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=Us, 
                                        biases=nce_biases, 
                                        labels=Ys, 
                                        inputs=embds, 
                                        num_sampled=neg_sample, 
                                        num_classes=vocab_size))
        
        norm = tf.sqrt(tf.reduce_sum(tf.square(Vs) , axis = 1))
        normalized_Vs = Vs/tf.reshape(norm , (-1,1))
        
        val_embds = tf.identity(normalized_Vs)
        similarity = tf.matmul(normalized_Vs , val_embds , transpose_b=True)
        
        return Xs , Ys , normalized_Vs , loss , similarity

In [10]:
def generator():
    yield from cbow_batch_generator.gen_batch(window_size , batch_size)

In [11]:
def run():
    with open ('indexWords.txt' , 'rt') as infile:
        index_words = eval(infile.read())
        
    data = tf.data.Dataset.from_generator(generator , (tf.int32 , tf.int32))
    iterator = data.make_initializable_iterator()
    context , target = iterator.get_next()
    
    Xs , Ys , Vs , loss , similarity = cbow()
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    with tf.Session() as sess:
        sess.run(iterator.initializer)
        sess.run(tf.global_variables_initializer())
        
        for i in range (steps):
            try:
                _,batch_loss = sess.run([optimizer , loss] , feed_dict={Xs: context.eval() , Ys:target.eval()})                    
                    
                if i%1000==0:
                    print(str(i)+":"+str(batch_loss))

                if i%5000 == 0:
                    sim = similarity.eval()

                    for j in val_indexes:
                        listt = (-sim[j,:]).argsort()[1:6]
                        print(listt)
                        print (index_words[j])
                        list2 = [index_words[k] for k in listt]
                        print (list2)

            except tf.errors.OutOfRangeError:
                sess.run(iterator.initializer)
                    

In [12]:
if __name__ == '__main__':
    run()

507079
0:264.62057
[ 4641 10804 27374 23815  7751]
two
['oversaw', 'bockris', 'ramstedt', 'commenting', 'nonfiction']
[16903 26011 32936 23532 27885]
person
['carving', 'fainter', 'baptism', 'exquisite', 'delarge']
[10937 15860 21109 22061 27328]
england
['sulfur', 'halides', 'caribou', 'mountaineers', 'defied']
[ 1713 16254 13726 23164   479]
week
['terror', 'rendition', 'mortis', 'woodcarving', 'periodical']
[16270 33507 28463 20373 32860]
big
['waves', 'unifem', 'blimps', 'fram', 'gemstone']
1000:53.26064
2000:48.278744
3000:15.223246
4000:20.46907
5000:53.030785
[1924  331  315 4634  275]
two
['autism', 'such', 'for', 'lincoln', 'nine']
[2093  141 7712 1924  205]
person
['behaviors', 'others', 'dagny', 'autism', 'modern']
[1678  639 1934 4743 1290]
england
['each', 'may', 'behavior', 'railroad', 'ayn']
[ 1713 16254 13726   479 23164]
week
['terror', 'rendition', 'mortis', 'periodical', 'woodcarving']
[9383  348 3006 8494 8597]
big
['turkish', 'involved', 'radiation', 'rio', 'myster

C:\Users\Fahimeh\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: generator 'gen_batch' raised StopIteration
  


507079
32000:6.954657
33000:7.2754064
34000:5.599559
35000:5.64413
[  253   275 10448 18525 13209]
two
['zero', 'nine', 'alexandria', 'lechuguilla', 'champions']
[ 1262  2093  9720 18262 20128]
person
['books', 'behaviors', 'separated', 'schottii', 'antelope']
[10937    93    47  2090    26]
england
['sulfur', 'or', 'by', 'talk', 'in']
[ 1713 15778 23594 23164 16254]
week
['terror', 'jupiter', 'shizuo', 'woodcarving', 'rendition']
[13660  9696  9383  2811  2368]
big
['pakanic', 'bodies', 'turkish', 'job', 'robot']
36000:5.5222154
37000:4.511359
38000:5.0852203
39000:4.086413
40000:6.712046
[  312 18525 18456 18702 13209]
two
['four', 'lechuguilla', 'geminiflora', 'shrevei', 'champions']
[ 1262  2093  9720 20128 18262]
person
['books', 'behaviors', 'separated', 'antelope', 'schottii']
[10937    93  2090  5594 32732]
england
['sulfur', 'or', 'talk', 'actor', 'threads']
[ 1713 15778 23594 23164 16254]
week
['terror', 'jupiter', 'shizuo', 'woodcarving', 'rendition']
[13660  9696  9383  236

111000:3.4547477
112000:4.3357706
113000:4.932417
114000:5.249297
115000:3.3857572
[ 7686 27911  8893 25502 19639]
two
['frequent', 'bumps', 'telephones', 'mummies', 'drove']
[ 1262 22384    15 21436 20128]
person
['books', 'sher', 'the', 'verner', 'antelope']
[10937  2090  2929 32732 11958]
england
['sulfur', 'talk', 'publicat', 'threads', 'ia']
[ 1713 23594  4292 11975 13726]
week
['terror', 'shizuo', 'entire', 'luma', 'mortis']
[13660 16119  2368 19920  4127]
big
['pakanic', 'jurisdiction', 'robot', 'boers', 'burn']
116000:4.313931
117000:2.2483156
118000:4.810199
119000:4.0679016
120000:4.700702
[18525 11922  8893 21723  7571]
two
['lechuguilla', 'aua', 'telephones', 'seinem', 'ludwig']
[ 1262 22384 20128  7204 21436]
person
['books', 'sher', 'antelope', 'naked', 'verner']
[10937  2090  2929 11958 32732]
england
['sulfur', 'talk', 'publicat', 'ia', 'threads']
[ 1713 23594  4292 13726 11975]
week
['terror', 'shizuo', 'entire', 'mortis', 'luma']
[13660 16119  2368 19920  4127]
big
['